In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

In [2]:
key_fd = open('../keys/gov_data_api_key.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

In [84]:
# datetime객체에서 문자열로 전처리해줘야 받아옴
start_date = '20201121'
end_date = '20201121'

In [87]:
corona_url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson"
url = f"{corona_url}?serviceKey={govapi_key}&pageNo=1&numOfRows=1&startCreateDt={start_date}&endCreateDt={end_date}"

In [88]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')

In [89]:
# 한 번 요청할 때 들어오는 결과 수
totalCount = soup.select_one('totalCount').text
totalCount

'189'

In [69]:
# 일자
date = soup.select('createDt')
# 지역명
continent = soup.select('areaNm')
# 국가명
country = soup.select('nationNm')
# 확진자 수
case = soup.select('natDefCnt')
# 사망자 수
death = soup.select('natDeathCnt')
# 치사률(확진자 대비 사망률)
death_rate = soup.select('natDeathRate')

In [70]:
# stdDay로 잘못 가져왔을 때
import re
a = '2020년 11월 21일 00시'
a = re.sub('[가-힣]+','-',a,0).strip()
a

'2020- 11- 21- 00-'

In [71]:
date_list = [] ; continent_list = [] ; country_list = [] ;case_list = [] ; death_list = [] ; death_rate_list = []

for i in date:
    date_list.append(parse(i.text).date())
for i in continent:
    continent_list.append(i.text)
for i in country:
    country_list.append(i.text)
for i in case:
    case_list.append(int(i.text))
for i in death:
    death_list.append(int(i.text))
for i in death_rate:
    death_rate_list.append(float(i.text))

In [72]:
df = pd.DataFrame({
    '일자':date_list,
    '대륙':continent_list, '국가':country_list,
    '확진자':case_list, '사망자':death_list, '사망률':death_rate_list
})
df

,일자,대륙,국가,확진자,사망자,사망률
0,2020-11-21,기타,기타지역 영토,194632,2371,1.218196
1,2020-11-21,아프리카,레소토,2058,44,2.137998
2,2020-11-21,아프리카,코모로,592,7,1.182432
3,2020-11-21,아프리카,상투메프린시페,974,16,1.642710
4,2020-11-21,아프리카,남수단,3016,59,1.956233
...,...,...,...,...,...,...
184,2020-11-21,아시아,마카오,46,0,0.000000
185,2020-11-21,아시아,대만,611,7,1.145663
186,2020-11-21,아시아,홍콩,5517,108,1.957586
187,2020-11-21,아시아,중국,86414,4634,5.362557


# 시작일

In [4]:
# 오늘 날짜
today = datetime.today()
march_2nd = datetime(2020,3,2)

# 날짜끼리의 사칙연산은 timedelta 클래스로 return됨, where properties such as days, seconds are available.
elapsed_count = (today - march_2nd).days
elapsed_count

295

In [5]:
elapsed_series = pd.date_range(march_2nd, periods= elapsed_count)
elapsed_series

DatetimeIndex(['2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
               '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11',
               ...
               '2020-12-12', '2020-12-13', '2020-12-14', '2020-12-15',
               '2020-12-16', '2020-12-17', '2020-12-18', '2020-12-19',
               '2020-12-20', '2020-12-21'],
              dtype='datetime64[ns]', length=295, freq='D')

# 요건 아직 트래픽 1000

In [6]:
date_list = [] ; continent_list = [] ; country_list = [] ;case_list = [] ; death_list = [] ; death_rate_list = []


corona_url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson"
    
for date in tqdm(elapsed_series):
    date =  date.strftime('%Y%m%d')


    url = f"{corona_url}?serviceKey={govapi_key}&pageNo=1&numOfRows=1&startCreateDt={date}&endCreateDt={date}"

    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')
    
    # 일자
    date = soup.select('createDt')
    # 지역명
    continent = soup.select('areaNm')
    # 국가명
    country = soup.select('nationNm')
    # 확진자 수
    case = soup.select('natDefCnt')
    # 사망자 수
    death = soup.select('natDeathCnt')
    # 치사률(확진자 대비 사망률)
    death_rate = soup.select('natDeathRate')

    for i in date:
        date_list.append(parse(i.text).date())
    for i in continent:
        continent_list.append(i.text)
    for i in country:
        country_list.append(i.text)
    for i in case:
        case_list.append(int(i.text))
    for i in death:
        death_list.append(int(i.text))
    for i in death_rate:
        death_rate_list.append(float(i.text))

100%|████████████████████████████████████████████████████████████████████████████████| 295/295 [05:05<00:00,  1.04s/it]


In [100]:
print(len(date_list))
print(len(continent_list))
print(len(country_list))
print(len(case_list))
print(len(death_list))
print(len(death_rate_list))

189
189
189
189
189
189


In [7]:
df = pd.DataFrame({
    '일자':date_list,
    '대륙':continent_list, '국가':country_list,
    '확진자':case_list, '사망자':death_list, '사망률':death_rate_list
})
df

,일자,대륙,국가,확진자,사망자,사망률
0,2020-03-16,기타,마요트,1,0,0.000000
1,2020-03-16,기타,퀴라소,2,0,0.000000
2,2020-03-16,기타,케이맨제도,1,0,0.000000
3,2020-03-16,기타,과들루프,3,0,0.000000
4,2020-03-16,기타,프랑스령레위니옹,6,0,0.000000
...,...,...,...,...,...,...
53398,2020-12-21,아시아,마카오,46,0,0.000000
53399,2020-12-21,아시아,대만,766,7,0.913838
53400,2020-12-21,아시아,홍콩,8152,130,1.594701
53401,2020-12-21,아시아,중국,86852,4634,5.335513


In [8]:
df.to_csv('./result/해외.csv',sep=',',encoding='utf8')